#Project summary notebook for report

In [106]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import json
 
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')


!pip install -q pyspark
!pip install findspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar
from inverted_index_colab import *



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u352-ga-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
--2023-01-05 22:03:40--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 18.165.83.25, 18.165.83.55, 18.165.83.74, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|18.165.83.25|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



## Build small inverted index from only trainQuery words

In [ ]:
from google.colab import auth
import signal

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)

try:
   auth.authenticate_user()
except: 
   pass
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP 
project_id = 'inforetassignment3'
!gcloud config set project {project_id}

data_bucket_name = '201640042_project'

Updated property [core/project].


In [ ]:
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
# can also just download all the bucket !gsutil -u {project_id} cp gs://{data_bucket_name}
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"

# for i in range(2, 27):
#   !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream{i}_preprocessed.parquet "wikidumps/"

Copying gs://201640042_project/multistream1_preprocessed.parquet...
- [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [ ]:
import findspark
findspark.init() 
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *



conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [ ]:
try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"
parquetFile = spark.read.parquet(path)
# parquetFile.show()

In [ ]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd
# doc_anchor_pairs = parquetFile.limit(1000).select("anchor_text", "id").rdd
# doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

# Opening JSON file
with open('queries_train.json') as json_file:
    data = json.load(json_file)
train_tokens = [token.group() for token in RE_WORD.finditer(' '.join(data.keys()).lower())]
train_filtered_vocab = set([tok for tok in train_tokens if (tok not in all_stopwords)])

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  filtered_tokens = [tok for tok in tokens if ((tok not in all_stopwords) and (tok in train_filtered_vocab))]
  doc_word_count = Counter(filtered_tokens)
  return [(tok, (id, tf)) for tok, tf in doc_word_count.items()]

In [95]:
res = doc_text_pairs.flatMap(lambda x: get_doc_len(x[0], x[1]))
doc_length = res.map(lambda x: (x[0], x[1]))
doc_vec_len = res.map(lambda x: (x[0], x[2]))
doc2len = doc_length.collectAsMap()
doc2vec_len = doc_vec_len.collectAsMap()

In [ ]:
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
posting_locs_list = partition_postings_and_write(postings_filtered).collect()

In [ ]:
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)

In [104]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# Count number of docs
inverted._N = parquetFile.count()
# Get each document vector legnth
inverted.doc2len = doc2len
inverted.doc2vec_len = doc2vec_len
# write the global stats out
inverted.write_index('.', 'train_body_index')

In [ ]:
# inverted = InvertedIndex().read_index('train_body_index')
# with open('queries_train.json') as json_file:
#     data = json.load(json_file)
# data.keys()

dict_keys(['best marvel movie', 'How do kids come to world?', 'Information retrieval', 'LinkedIn', 'How to make coffee?', 'Ritalin', 'How to make wine at home?', 'Most expensive city in the world', 'India', 'how to make money fast?', 'Netflix', 'Apple computer', 'The Simpsons', 'World cup', 'How to lose weight?', 'Java', 'Air Jordan', 'how to deal with depression?', 'How do you make gold', 'Marijuana', 'How to make hummus', 'Winter', 'Rick and Morty', 'Natural Language processing', 'World Cup 2022', 'Dolly the sheep', 'Ciggarets', 'What is the best place to live in?', 'Elon musk', 'How do you breed flowers?'])

In [103]:
super_posting_locs

defaultdict(list,
            {'world': [('56_000.bin', 0)],
             'best': [('87_000.bin', 0)],
             'live': [('12_000.bin', 0)],
             'place': [('66_000.bin', 0)],
             'make': [('51_000.bin', 0)],
             'air': [('72_000.bin', 0)],
             'information': [('11_000.bin', 0)],
             'processing': [('5_000.bin', 0)],
             'apple': [('5_000.bin', 486)],
             'language': [('21_000.bin', 0)],
             'natural': [('24_000.bin', 0)],
             'depression': [('115_000.bin', 0)],
             'home': [('63_000.bin', 0)],
             'weight': [('93_000.bin', 0)],
             'lose': [('89_000.bin', 0)],
             'deal': [('27_000.bin', 0)],
             'winter': [('17_000.bin', 0)],
             'come': [('8_000.bin', 0)],
             'cup': [('28_000.bin', 0)],
             'city': [('104_000.bin', 0)],
             'fast': [('104_000.bin', 2448)],
             'money': [('73_000.bin', 0)],
             'compute